# Import Libraries

In [1]:
import pandas as pd, numpy as np #basic package
from scipy import stats
from sklearn import preprocessing

from sklearn.ensemble import BaggingClassifier #bagging
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

# Data Preprocessing 

### Transformation of labels

In [2]:
data = pd.read_csv('diabetic_data.csv')

#table of readmission status column
data.readmitted.value_counts()

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64

In [3]:
readmitted_tran = np.zeros(len(data.readmitted))
readmitted_tran[data.readmitted == "<30"] = 1

#trasfer to series and then add to data frame
readmitted_tran = pd.Series(readmitted_tran)
data['readmitted_tran'] = readmitted_tran

### Other Variable checking

In [4]:
list(data)

['encounter_id',
 'patient_nbr',
 'race',
 'gender',
 'age',
 'weight',
 'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'payer_code',
 'medical_specialty',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'diag_1',
 'diag_2',
 'diag_3',
 'number_diagnoses',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed',
 'readmitted',
 'readmitted_tran']

In [6]:
for i in range(0,51):
    print(data.iloc[:,i].value_counts())

96210942     1
89943846     1
384306986    1
94650156     1
83156784     1
2674482      1
281345844    1
193616274    1
355508024    1
165973818    1
125278944    1
420873188    1
157241154    1
161161032    1
174855390    1
134950734    1
154128210    1
96993108     1
122064144    1
297770840    1
382612616    1
165134172    1
108244830    1
210578766    1
443842340    1
151469730    1
289146210    1
154590960    1
145948404    1
176328594    1
            ..
249722520    1
111830682    1
126506652    1
80193186     1
186881700    1
147162726    1
263120844    1
249665124    1
151295556    1
113303472    1
73909806     1
422050106    1
13655088     1
168523320    1
296140568    1
98784828     1
157333056    1
280536642    1
130655706    1
190162530    1
107017800    1
103828530    1
176744010    1
172279374    1
297285200    1
74454612     1
208073976    1
166229592    1
38340702     1
77856768     1
Name: encounter_id, Length: 101766, dtype: int64
88785891     40
43140906     28
2319

None    96420
Norm     2597
>200     1485
>300     1264
Name: max_glu_serum, dtype: int64
None    84748
>8       8216
Norm     4990
>7       3812
Name: A1Cresult, dtype: int64
No        81778
Steady    18346
Up         1067
Down        575
Name: metformin, dtype: int64
No        100227
Steady      1384
Up           110
Down          45
Name: repaglinide, dtype: int64
No        101063
Steady       668
Up            24
Down          11
Name: nateglinide, dtype: int64
No        101680
Steady        79
Up             6
Down           1
Name: chlorpropamide, dtype: int64
No        96575
Steady     4670
Up          327
Down        194
Name: glimepiride, dtype: int64
No        101765
Steady         1
Name: acetohexamide, dtype: int64
No        89080
Steady    11356
Up          770
Down        560
Name: glipizide, dtype: int64
No        91116
Steady     9274
Up          812
Down        564
Name: glyburide, dtype: int64
No        101743
Steady        23
Name: tolbutamide, dtype: int64
No       

### Drop Variables Visually

In [7]:
data.replace('?', np.nan, inplace = True)
data1 = data.copy()
del data1["encounter_id"]
del data1["patient_nbr"]
del data1["acetohexamide"]
del data1["troglitazone"]
del data1["examide"]
del data1["citoglipton"]
del data1["glimepiride-pioglitazone"]
del data1["metformin-rosiglitazone"]
del data1["metformin-pioglitazone"]
del data1["readmitted"]
data1.shape

(101766, 41)

## Prepare Training vs Test Set

### Seperate Target

In [8]:
target = data1["readmitted_tran"]
df = data1.drop('readmitted_tran',axis = 1)

## Missing Values

In [9]:
df.isnull().any()[df.isnull().any() == True]

race                 True
weight               True
payer_code           True
medical_specialty    True
diag_1               True
diag_2               True
diag_3               True
dtype: bool

In [10]:
df['race'].replace(np.nan, 0, inplace = True)
df['weight'].replace(np.nan, 0, inplace = True)
df['payer_code'].replace(np.nan, 0, inplace = True)
df['medical_specialty'].replace(np.nan, 0, inplace = True)
df['diag_1'].replace(np.nan, 0, inplace = True)
df['diag_2'].replace(np.nan, 0, inplace = True)
df['diag_3'].replace(np.nan, 0, inplace = True)

### Dummy Variables

### Race

In [11]:
race_dummy = pd.get_dummies(df['race'])
race_dummy.head()

,0,AfricanAmerican,Asian,Caucasian,Hispanic,Other
0,0,0,0,1,0,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,1,0,0
4,0,0,0,1,0,0


### Gender

In [12]:
gender_dummy = pd.get_dummies(df['gender'])
gender_dummy.head()

,Female,Male,Unknown/Invalid
0,1,0,0
1,1,0,0
2,1,0,0
3,0,1,0
4,0,1,0


### age

In [13]:
age_dummy = pd.get_dummies(df['age'])
age_dummy.head()

,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100)
0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0


### weight

In [14]:
weight_dummy = pd.get_dummies(df['weight'])
weight_dummy.head()

,0,>200,[0-25),[100-125),[125-150),[150-175),[175-200),[25-50),[50-75),[75-100)
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0


### admission_type_id

In [15]:
admission_type_id_dummy = pd.get_dummies(df['admission_type_id'])
admission_type_id_dummy.head()

,1,2,3,4,5,6,7,8
0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0


### discharge_disposition_id

In [16]:
discharge_disposition_id_dummy = pd.get_dummies(df['discharge_disposition_id'])
discharge_disposition_id_dummy.head()

,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,22,23,24,25,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### admission_source_id

In [17]:
admission_source_id_dummy = pd.get_dummies(df['admission_source_id'])
admission_source_id_dummy.head()

,1,2,3,4,5,6,7,8,9,10,11,13,14,17,20,22,25
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


### payer_code

In [18]:
payer_code_dummy = pd.get_dummies(df['payer_code'])
payer_code_dummy.head()

,0,BC,CH,CM,CP,DM,FR,HM,MC,MD,MP,OG,OT,PO,SI,SP,UN,WC
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### medical_specialty

In [19]:
medical_specialty_dummy = pd.get_dummies(df['medical_specialty'])
medical_specialty_dummy.head()

,0,AllergyandImmunology,Anesthesiology,Anesthesiology-Pediatric,Cardiology,Cardiology-Pediatric,DCPTEAM,Dentistry,Dermatology,Emergency/Trauma,...,Surgery-General,Surgery-Maxillofacial,Surgery-Neuro,Surgery-Pediatric,Surgery-Plastic,Surgery-PlasticwithinHeadandNeck,Surgery-Thoracic,Surgery-Vascular,SurgicalSpecialty,Urology
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### diag_3

In [20]:
diag_3_dummy = pd.get_dummies(df['diag_3'])
diag_3_dummy.head()

,0,11,110,111,112,115,117,122,123,131,...,V61,V62,V63,V64,V65,V66,V70,V72,V85,V86
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### max_glu_serum

In [21]:
max_glu_serum_dummy = pd.get_dummies(df['max_glu_serum'])
max_glu_serum_dummy.head()

,>200,>300,None,Norm
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


### A1Cresult

In [22]:
A1Cresult_dummy = pd.get_dummies(df['A1Cresult'])
A1Cresult_dummy.head()

,>7,>8,None,Norm
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


### metformin

In [23]:
metformin_dummy = pd.get_dummies(df['metformin'])
metformin_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### repaglinide

In [24]:
repaglinide_dummy = pd.get_dummies(df['repaglinide'])
repaglinide_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### nateglinide

In [25]:
nateglinide_dummy = pd.get_dummies(df['nateglinide'])
nateglinide_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### chlorpropamide

In [26]:
chlorpropamide_dummy = pd.get_dummies(df['chlorpropamide'])
chlorpropamide_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### glimepiride

In [27]:
glimepiride_dummy = pd.get_dummies(df['glimepiride'])
glimepiride_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### glipizide

In [28]:
glipizide_dummy = pd.get_dummies(df['glipizide'])
glipizide_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,0,1,0
3,0,1,0,0
4,0,0,1,0


### glyburide

In [29]:
glyburide_dummy = pd.get_dummies(df['glyburide'])
glyburide_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### tolbutamide

In [30]:
tolbutamide_dummy = pd.get_dummies(df['tolbutamide'])
tolbutamide_dummy.head()

,No,Steady
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


### pioglitazone

In [31]:
pioglitazone_dummy = pd.get_dummies(df['pioglitazone'])
pioglitazone_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### rosiglitazone

In [32]:
rosiglitazone_dummy = pd.get_dummies(df['rosiglitazone'])
rosiglitazone_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### acarbose

In [33]:
acarbose_dummy = pd.get_dummies(df['acarbose'])
acarbose_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### miglitol

In [34]:
miglitol_dummy = pd.get_dummies(df['miglitol'])
miglitol_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### tolazamide

In [35]:
tolazamide_dummy = pd.get_dummies(df['tolazamide'])
tolazamide_dummy.head()

,No,Steady,Up
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


### insulin

In [36]:
insulin_dummy = pd.get_dummies(df['insulin'])
insulin_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,0,0,1
2,0,1,0,0
3,0,0,0,1
4,0,0,1,0


### glyburide-metformin

In [37]:
glyburide_metformin_dummy = pd.get_dummies(df['glyburide-metformin'])
glyburide_metformin_dummy.head()

,Down,No,Steady,Up
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


### glipizide-metformin

In [38]:
glipizide_metformin_dummy = pd.get_dummies(df['glipizide-metformin'])
glipizide_metformin_dummy.head()

,No,Steady
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


### change

In [39]:
change_dummy = pd.get_dummies(df['change'])
change_dummy.head()

,Ch,No
0,0,1
1,1,0
2,0,1
3,1,0
4,1,0


### diabetesMed

In [40]:
diabetesMed_dummy = pd.get_dummies(df['diabetesMed'])
diabetesMed_dummy.head()

,No,Yes
0,1,0
1,0,1
2,0,1
3,0,1
4,0,1


## Merge

### Drop Categorical Variable

In [41]:
dft = df.drop("race",axis=1)
dft = dft.drop("gender",axis=1)
dft = dft.drop("age",axis=1)
dft = dft.drop("weight",axis=1)
dft = dft.drop("admission_type_id",axis=1)
dft = dft.drop("discharge_disposition_id",axis=1)
dft = dft.drop("admission_source_id",axis=1)
dft = dft.drop("payer_code",axis=1)
dft = dft.drop("medical_specialty",axis=1)
dft = dft.drop("diag_3",axis=1)
dft = dft.drop("max_glu_serum",axis=1)
dft = dft.drop("A1Cresult",axis=1)
dft = dft.drop("metformin",axis=1)
dft = dft.drop("repaglinide",axis=1)
dft = dft.drop("nateglinide",axis=1)
dft = dft.drop("chlorpropamide",axis=1)
dft = dft.drop("glimepiride",axis=1)
dft = dft.drop("glipizide",axis=1)
dft = dft.drop("glyburide",axis=1)
dft = dft.drop("tolbutamide",axis=1)
dft = dft.drop("pioglitazone",axis=1)
dft = dft.drop("rosiglitazone",axis=1)
dft = dft.drop("acarbose",axis=1)
dft = dft.drop("miglitol",axis=1)
dft = dft.drop("tolazamide",axis=1)
dft = dft.drop("insulin",axis=1)
dft = dft.drop("glyburide-metformin",axis=1)
dft = dft.drop("glipizide-metformin",axis=1)
dft = dft.drop("change",axis=1)
dft = dft.drop("diabetesMed",axis=1)

In [42]:
dfs = pd.concat([race_dummy,gender_dummy,age_dummy,weight_dummy,admission_type_id_dummy,discharge_disposition_id_dummy,
      admission_source_id_dummy,payer_code_dummy,medical_specialty_dummy,diag_3_dummy,max_glu_serum_dummy,
      A1Cresult_dummy,metformin_dummy,repaglinide_dummy,nateglinide_dummy,chlorpropamide_dummy,glimepiride_dummy,
      glipizide_dummy,glyburide_dummy,tolbutamide_dummy,pioglitazone_dummy,rosiglitazone_dummy,acarbose_dummy,miglitol_dummy,
      tolazamide_dummy,insulin_dummy,glipizide_metformin_dummy,glyburide_metformin_dummy,change_dummy,diabetesMed_dummy],axis=1)
df = pd.concat([dft,dfs],axis=1)


In [43]:
df.shape

(101766, 1042)

## Normalization

### Make sure all ordinal variables are numerical type

In [44]:
numeric_list = ["time_in_hospital","num_lab_procedures","num_procedures","num_medications",
               "number_outpatient","number_emergency","number_inpatient","number_diagnoses"]

In [45]:
df[numeric_list].apply(pd.to_numeric)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,1,41,0,1,0,0,0,1
1,3,59,0,18,0,0,0,9
2,2,11,5,13,2,0,1,6
3,2,44,1,16,0,0,0,7
4,1,51,0,8,0,0,0,5
5,3,31,6,16,0,0,0,9
6,4,70,1,21,0,0,0,7
7,5,73,0,12,0,0,0,8
8,13,68,2,28,0,0,0,8
9,12,33,3,18,0,0,0,8


### Normalize Numerical Variable

In [46]:
def z_norm(x):
    
    ret = (x - x.mean()) / x.std()
    
    return ret

In [47]:
x = df[numeric_list]
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

In [48]:
df = df.drop(numeric_list,axis = 1)

In [49]:
df = pd.concat([df,x],axis=1)

In [50]:
df.shape

(101766, 1042)

## Train+Test Data Prepare

In [51]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2)

In [52]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(81412, 1042) (81412,)
(20354, 1042) (20354,)


In [53]:
test_set = pd.concat([X_test,y_test],axis=1)
test_set.to_csv("test_set.csv")

In [54]:
X_train.to_csv("train.csv")
y_train.to_csv("train_target.csv")

## Import Data and Define Global Variables

In [54]:
X_train = pd.read_csv("train.csv")
y_train = pd.read_csv("train_target.csv")
test = pd.read_csv("test_set.csv")

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
def top_40_plus(model, X, y):

    rfe = RFE(model, 40)
    fit = rfe.fit(X, y)
    
    keep = []
    for i,a in enumerate(fit.support_):
        if (a == True):
            keep.append(i)
    
    return keep

In [56]:
def lr(X_train, y_train, test):

    lr = BaggingClassifier(base_estimator = LogisticRegression())
    lr.fit(X_train, y_train)
    y_pred = lr.predict_proba(X_test)
    
    return y_pred

In [58]:
def nb(X_train, y_train, test):
    
    nb = BaggingClassifier(base_estimator = GaussianNB())
    nb.fit(X_train,y_train)
    y_pred = nb.predict_proba(X_test)
    
    return y_pred

In [59]:
def rf(X_train, y_train, test):
    
    rf = BaggingClassifier(base_estimator = RandomForestClassifier())
    rf.fit(X_train,y_train)
    y_pred = rf.predict_proba(X_test)
    
    return y_pred

In [60]:
def sv(X_train, y_train, test):
    
    sv = BaggingClassifier(base_estimator = SVC(probability = True))
    sv.fit(X_train,y_train)
    y_pred = sv.predict_proba(X_test)
    
    return y_pred

In [61]:
def gb(X_train, y_train, test):

    gb = BaggingClassifier(base_estimator = GradientBoostingClassifier())
    gb.fit(X_train, y_train)
    y_pred = gb.predict_proba(X_test)
    
    return y_pred

In [62]:
def ensemble(X_train, y_train, X_test):
    lr_keep = top_40_plus(LogisticRegression(), X_train, y_train)

    # Declared Globally
    # X_train, X_test, y_train, y_test 
   
    lr_y = lr(X_train.iloc[:,lr_keep], y_train, X_test.iloc[:,lr_keep])
    sv_y = sv(X_train.iloc[:,lr_keep], y_train, X_test.iloc[:,lr_keep])
    rf_y = rf(X_train.iloc[:,lr_keep], y_train, X_test.iloc[:,lr_keep])
    nb_y = nb(X_train.iloc[:,lr_keep], y_train, X_test.iloc[:,lr_keep])
    gb_y = gb(X_train.iloc[:,lr_keep], y_train, X_test.iloc[:,lr_keep])
    
    final_y = []
    
    for i in range(len(X_test)):
        final_y.append((lr_y[i][1]+ nb_y[i][1]+rf_y[i][1]+ sv_y[i][1]+ gb_y[i][1])/5)
        
    return final_y

In [63]:
e = ensemble(X_train, y_train, X_test)

ValueError: could not convert string to float: 'V15'